# How to deal with a Nuclear Shell Model (NSM) Hamiltonian -a funny guide to madness-

NSM Hamiltonians are many-body Hamiltonians represented in a single particle framework (or nucleon modes) described by Valence orbitals (with the famous set of quantum numbers that describe the 3D harmonic oscillator $|n,l,j,m,t,t_z\rangle$. The last two quantum numbers describe the isospin of the nucleons (proton or neutron).

First of all, we need to commit: we need to decide which Valence shell we are going to use. This choice translates in selecting the proper "nuclear_interaction.txt" file related to the correspoding valence shell.

- p shell --> 'data/cki' file 
- sd shell --> 'data/usdb.nat' file 
- pf shell --> 'data/gxpf1a' file 

Don't worry about these names, for you these simply are file text with matrix entries that are going to be the parameters of your NSM Hamiltonian:
$$
H=\sum_a c^{+}_a c_a + \frac{1}{4}\sum_{abcd} v_{abdc} c^{+}_a c^{+}_b c_c c_d.
$$

Ok, too much verbose, let's start with the Imports

In [1]:
from src.NSMFermions.hamiltonian_utils import FermiHubbardHamiltonian,FermiHubbardHamiltonianOptimized # the many-body Hamiltonian class
from src.NSMFermions.nuclear_physics_utils import SingleParticleState,J2operator,get_twobody_nuclearshell_model # routines and class useful for the nuclear part
import scipy # just scipy, easy, no?
import numpy as np 
import matplotlib.pyplot as plt # to plot things

/home/ecosta/miniconda3/envs/quantum_annealing_nsm/lib/python3.10/site-packages/tqdm_joblib/__init__.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


We initialize the Single Particle State class with the corresponding file text. We also initialize the number of proton neutron and the corresponding number of modes for each nucleon

In [2]:
file_name='data/cki'

SPS=SingleParticleState(file_name=file_name)
# single particle energies
print('single particle energies=',SPS.energies,'\n')
print('mapping between nucleon modes a and the quantum numbers',SPS.state_encoding)

nucleon_modes_per_isospin=SPS.energies.shape[0]//2 # we are counting per species. I know, we need a .num_modes attribute

num_neutrons=2
num_protons=2

single particle energies= [1.63 1.63 1.63 1.63 2.27 2.27 1.63 1.63 1.63 1.63 2.27 2.27] 

mapping between nucleon modes a and the quantum numbers [(0, 1, 1.5, np.float64(-1.5), 0.5, 0.5), (0, 1, 1.5, np.float64(-0.5), 0.5, 0.5), (0, 1, 1.5, np.float64(0.5), 0.5, 0.5), (0, 1, 1.5, np.float64(1.5), 0.5, 0.5), (0, 1, 0.5, np.float64(-0.5), 0.5, 0.5), (0, 1, 0.5, np.float64(0.5), 0.5, 0.5), (0, 1, 1.5, np.float64(-1.5), 0.5, -0.5), (0, 1, 1.5, np.float64(-0.5), 0.5, -0.5), (0, 1, 1.5, np.float64(0.5), 0.5, -0.5), (0, 1, 1.5, np.float64(1.5), 0.5, -0.5), (0, 1, 0.5, np.float64(-0.5), 0.5, -0.5), (0, 1, 0.5, np.float64(0.5), 0.5, -0.5)]


Now, we can initialize the NSM Hamiltonian and look at the many-body basis. It is a tensor with the first index as the many-body basis index and second index the corresponding nucleon modes in the tensor product basis state

In [3]:
NSMHamiltonian=FermiHubbardHamiltonianOptimized(size_a=nucleon_modes_per_isospin,size_b=nucleon_modes_per_isospin,nparticles_a=num_neutrons,nparticles_b=num_protons,symmetries=[SPS.total_M_zero]) # the symmetry that we need is the M=0 condition, if we do not add anything we get the full many-body basis

print(NSMHamiltonian.basis)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1mUntyped global name 'comb':[0m [1m[1mCannot determine Numba type of <class 'builtin_function_or_method'>[0m
[1m
File "src/NSMFermions/numba_utils.py", line 12:[0m
[1mdef generate_combinations(n, k):
[1m    n_comb = comb(n, k)  # now OK
[0m    [1m^[0m[0m
[0m
[0m[1mDuring: Pass nopython_type_inference[0m
[0m[1mDuring: resolving callee type: type(CPUDispatcher(<function generate_combinations at 0x7fc829beb5b0>))[0m
[0m[1mDuring: typing of call at /home/ecosta/QuantumMachineLearningFermions/src/NSMFermions/numba_utils.py (40)[0m
[0m[1mDuring: typing of call at /home/ecosta/QuantumMachineLearningFermions/src/NSMFermions/numba_utils.py (41)[0m
[1m
File "src/NSMFermions/numba_utils.py", line 40:[0m
[1mdef generate_bit_basis_numba(nsites_a, nparticles_a, nsites_b, nparticles_b):
[1m    combos_a = generate_combina

Now we initialize the NSM Hamiltonian, starting from the external potential $\sum_a e_a c^{+}_a c_a$

In [4]:
NSMHamiltonian.get_external_potential_optimized(external_potential=SPS.energies)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at /home/ecosta/QuantumMachineLearningFermions/src/NSMFermions/fermi_hubbard_library.py (717)[0m
[1m
File "src/NSMFermions/fermi_hubbard_library.py", line 717:[0m
[1m    def _adag_adag_a_a_loop_numba_with_dict(
        <source elided>

[1m    @njit
[0m    [1m^[0m[0m

[0m[1mDuring: Pass nopython_type_inference[0m 

This error may have been caused by the following argument(s):
- argument 0: [1mCannot determine Numba type of <class 'src.NSMFermions.hamiltonian_utils.FermiHubbardHamiltonian'>[0m


In [24]:
print(NSMHamiltonian.external_potential)

None


In [16]:
twobody_dict,_=get_twobody_nuclearshell_model(file_name=file_name)


Computing the matrix, pls wait... (u_u) 



100%|██████████| 24/24 [00:46<00:00,  1.95s/it]


In [26]:

NSMHamiltonian.get_twobody_interaction_optimized(twobody_dict)



Building two-body operator with 16496 terms...


100%|██████████| 16496/16496 [00:12<00:00, 1363.34it/s]


✅ Two-body operator built: shape=(28503, 28503), nnz=6030191


In order to use the hamiltonian, we need to compute the full Hamiltonian matrix, using get_hamiltonian()

In [27]:
NSMHamiltonian.get_hamiltonian()

In [28]:
print(NSMHamiltonian.twobody_operator)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6030191 stored elements and shape (28503, 28503)>
  Coords	Values
  (0, 0)	-35.723430178571405
  (0, 1)	0.10435216049167698
  (0, 2)	0.8598999999999996
  (0, 3)	0.3596601225771707
  (0, 4)	-0.19837717572588723
  (0, 5)	0.10165680416643712
  (0, 6)	-0.6918637014569978
  (0, 7)	0.0683045371787489
  (0, 8)	-0.2530987765796636
  (0, 9)	-0.4546230489830741
  (0, 10)	0.4237229179758751
  (0, 11)	-0.10165680416643709
  (0, 13)	-0.6918637014569978
  (0, 15)	-0.19515286510161337
  (0, 16)	-0.14860413794393482
  (0, 17)	-1.0872781580010684
  (0, 18)	0.23293633710277387
  (0, 19)	-0.6489247535071269
  (0, 20)	0.9007818899896537
  (0, 21)	0.09673599074161547
  (0, 22)	0.09673599074161547
  (0, 23)	-1.3995613925794361
  (0, 24)	-0.07222039837222677
  (0, 25)	0.1274540877860897
  (0, 26)	-1.5803666666666671
  :	:
  (28502, 28467)	0.1295371825641837
  (28502, 28468)	0.25907436512836757
  (28502, 28469)	-5.551115123125783e-17
  (28502, 2847

In [29]:
print(NSMHamiltonian.adag_adag_a_a_matrix_optimized(1,17,1,17))



<COOrdinate sparse matrix of dtype 'float64'
	with 3582 stored elements and shape (28503, 28503)>
  Coords	Values
  (0, 0)	-1.0
  (1, 1)	-1.0
  (2, 2)	-1.0
  (3, 3)	-1.0
  (4, 4)	-1.0
  (5, 5)	-1.0
  (6, 6)	-1.0
  (7, 7)	-1.0
  (8, 8)	-1.0
  (11, 11)	-1.0
  (12, 12)	-1.0
  (13, 13)	-1.0
  (14, 14)	-1.0
  (15, 15)	-1.0
  (16, 16)	-1.0
  (17, 17)	-1.0
  (18, 18)	-1.0
  (20, 20)	-1.0
  (21, 21)	-1.0
  (22, 22)	-1.0
  (23, 23)	-1.0
  (24, 24)	-1.0
  (25, 25)	-1.0
  (26, 26)	-1.0
  (27, 27)	-1.0
  :	:
  (16578, 16578)	-1.0
  (16579, 16579)	-1.0
  (16580, 16580)	-1.0
  (16584, 16584)	-1.0
  (16585, 16585)	-1.0
  (16586, 16586)	-1.0
  (16588, 16588)	-1.0
  (16589, 16589)	-1.0
  (16592, 16592)	-1.0
  (16593, 16593)	-1.0
  (16596, 16596)	-1.0
  (16600, 16600)	-1.0
  (16601, 16601)	-1.0
  (16602, 16602)	-1.0
  (16603, 16603)	-1.0
  (16604, 16604)	-1.0
  (16610, 16610)	-1.0
  (16611, 16611)	-1.0
  (16616, 16616)	-1.0
  (16625, 16625)	-1.0
  (16626, 16626)	-1.0
  (16627, 16627)	-1.0
  (16641, 1664

At this point, getting the spectrum is easy as getting the empadronamiento in Barcelona (joke)

In [30]:
egs,psigs=NSMHamiltonian.get_spectrum(n_states=1) #we are only interested in the gs 

print(f'energy ground state={egs[0]:.5} Mev \n')


energy ground state=-92.775 Mev 



We can also play with other attributes that the Hamiltonian can provide, such as 2-body operators $T_{ab}^{cd}=c^{+}_a c^{+}_{b} c_c c_d$

In [20]:
t_0312=NSMHamiltonian.adag_adag_a_a_matrix(0,3,1,2) # it's a matrix in the many-body basis

# we compute the expectation value using Scipy <psi| T_01^23 |psi>
expectation_value=psigs[:,0].conjugate().dot(t_0312.dot(psigs[:,0])) # the 0 index is because  psigs \in [dim(Hilbert space),n_states]
print('expectation value=',expectation_value)

expectation value= 0.19708205121236266


To call the Hamiltonian we simply use

In [ ]:
print(NSMHamiltonian.hamiltonian)